# website: https://pypi.org/project/ai4bharat-transliteration/

In [ ]:
!pip install ai4bharat-transliteration==1.1.3

In [1]:
import pandas as pd
import numpy as np
import glob
from tqdm.auto import tqdm
import re
import json
pd.set_option("display.max_columns",None)

/home/hp/election_analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Supported languages and their codes

{'as': ['Assamese - অসমীয়া'],
 'bn': ['Bangla - বাংলা'],
 'brx': ['Boro - बड़ो'],
 'gu': ['Gujarati - ગુજરાતી'],
 'hi': ['Hindi - हिंदी'],
 'kn': ['Kannada - ಕನ್ನಡ'],
 'ks': ['Kashmiri - كٲشُر'],
 'gom': ['Konkani Goan - कोंकणी'],
 'mai': ['Maithili - मैथिली'],
 'ml': ['Malayalam - മലയാളം'],
 'mni': ['Manipuri - ꯃꯤꯇꯩꯂꯣꯟ'],
 'mr': ['Marathi - मराठी'],
 'ne': ['Nepali - नेपाली'],
 'or': ['Oriya - ଓଡ଼ିଆ'],
 'pa': ['Panjabi - ਪੰਜਾਬੀ'],
 'sa': ['Sanskrit - संस्कृतम्'],
 'sd': ['Sindhi - سنڌي'],
 'si': ['Sinhala - සිංහල'],
 'ta': ['Tamil - தமிழ்'],
 'te': ['Telugu - తెలుగు'],
 'ur': ['Urdu - اُردُو']}

{'as': ['Assamese - অসমীয়া'],
 'bn': ['Bangla - বাংলা'],
 'brx': ['Boro - बड़ो'],
 'gu': ['Gujarati - ગુજરાતી'],
 'hi': ['Hindi - हिंदी'],
 'kn': ['Kannada - ಕನ್ನಡ'],
 'ks': ['Kashmiri - كٲشُر'],
 'gom': ['Konkani Goan - कोंकणी'],
 'mai': ['Maithili - मैथिली'],
 'ml': ['Malayalam - മലയാളം'],
 'mni': ['Manipuri - ꯃꯤꯇꯩꯂꯣꯟ'],
 'mr': ['Marathi - मराठी'],
 'ne': ['Nepali - नेपाली'],
 'or': ['Oriya - ଓଡ଼ିଆ'],
 'pa': ['Panjabi - ਪੰਜਾਬੀ'],
 'sa': ['Sanskrit - संस्कृतम्'],
 'sd': ['Sindhi - سنڌي'],
 'si': ['Sinhala - සිංහල'],
 'ta': ['Tamil - தமிழ்'],
 'te': ['Telugu - తెలుగు'],
 'ur': ['Urdu - اُردُو']}

In [3]:
from ai4bharat.transliteration import XlitEngine

# English to regional language

In [4]:
# Read file with data to convert
# File should only contain columns that you want to convert/transliterate
filename = '/home/hp/Downloads/20_Remaining State - Translation_Zila.xlsx'
df_main = pd.read_excel(filename)

In [5]:
# Perform the necessary cleaning here

In [6]:
# collecting words to translate (to be fed into transliteration model)

words = []
for i in tqdm(df_main.columns): #iterating on columns
    i = str(i)
    df_temp = df_main[df_main[i].notnull()][[i]].drop_duplicates() #drop duplicate rows
    df_temp[i] = df_temp[i].apply(lambda x:x.lower().strip().split(' ')) #converting every word to lower and removing spaces
    df_temp = df_temp.explode([i]) # converting list to rows
    words.append(df_temp[i].unique().tolist()) # taking only unique from the column

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 200.63it/s]


In [7]:
unique_words = list(set([i for sublist in words for i in sublist]))

In [8]:
# Load transliteration object to ram
engine = XlitEngine("hi", beam_width=4, rescore=True) #'hi' stands for Hindi

Initializing Multilingual model for transliteration


Loading dicts into RAM: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.20s/it]


In [9]:
# load the json of already converted names
with open ('/home/hp/Downloads/eng_to_hin.json', "r") as file:
    words_dict = json.load(file)

In [10]:
print(len(words_dict.keys()))

90996


In [11]:
def transliteration(unique_words,words_dict):
    converted_words = []
    for i in tqdm(unique_words):
        try:
            converted_words.append(words_dict[i][0])
            continue
        except:
            pass
        i = i + ' '
        temp = ''
        converted_words_temp = ''
        for letter in i: #check each letter if it is an alphabet
            if(letter.isalpha()):
                temp = temp + letter #string till we find alphabet
            else:
                try:
                    try:
                        converted_words_temp = converted_words_temp + words_dict[temp.strip()][0] + str(letter)
                    except:
                        #word not found in words_dict
                        temp_eng2lang = engine.translit_word(temp.strip(), topk=1)
                        words_dict[temp.strip()] = temp_eng2lang['hi'][0].replace('\u200c','') # add word to dictinoary 
                        converted_words_temp = converted_words_temp + words_dict[temp.strip()] + str(letter)
                except:
                    #exceptional case: leave the word as it is
                    converted_words_temp = converted_words_temp + str(letter)
                temp = ''
        converted_words.append(converted_words_temp.strip())
    return converted_words

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 954/954 [01:27<00:00, 10.91it/s]


In [12]:
converted_words_dict = dict(zip(unique_words, converted_words))

In [13]:
for i in tqdm(df_main.columns):
    df_main[str(i) + '_converted'] = df_main[str(i)].apply(lambda x:np.nan if(x!=x) else ' '.join([converted_words_dict[i] for i in x.lower().strip().split(' ')]))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 605.68it/s]


In [14]:
# overwrite the dictionary
with open('eng_to_hin.json', "w") as outfile:
    json.dump(words_dict, outfile)

In [15]:
# save converted file
final_filename = filename.split('.')[0] + '_converted.' + filename.split('.')[1] 
df_main.to_excel(final_filename, index=False)

# Regional to english

In [ ]:
# Read file with data to convert
# File should only contain columns that you want to convert/transliterate
filename = r'file.xlsx'

df = pd.read_excel(filename)

In [ ]:
# Perform the necessary cleaning here

In [ ]:
engine = XlitEngine(src_script_type="indic", beam_width=4, rescore=True)

In [ ]:
# collecting words to translate (to be fed into transliteration model)

words = []
for i in tqdm(df_main.columns): #iterating on columns
    i = str(i)
    df_temp = df_main[df_main[i].notnull()][[i]].drop_duplicates() #drop duplicate rows
    df_temp[i] = df_temp[i].apply(lambda x:x.lower().strip().split(' ')) #converting every word to lower and removing spaces
    df_temp = df_temp.explode([i]) # converting list to rows
    words.append(df_temp[i].unique().tolist()) # taking only unique from the column

In [ ]:
unique_words = list(set([i for sublist in words for i in sublist]))

In [ ]:
# load the json of already converted names
with open ('hin_to_eng.json', "r") as file:
    words_dict = json.load(file)

In [ ]:
l = []
for i in tqdm(l1):
    try:
        l.append(diss[i][0])
        continue
    except:
        pass
    l.append(e.translit_word(i, lang_code="hi", topk=1)[0])  # 'hi' language code for hindi

In [ ]:
df1 = pd.DataFrame()
df1['hi'] = l1
df1['eng'] = l

In [ ]:
my_dict = df1.set_index('hi')['eng'].to_dict()

In [ ]:
for i in tqdm(df.columns):
    df[str(i) + '_converted'] = df[str(i)].apply(lambda x:np.nan if(x!=x) else ' '.join([my_dict[i] for i in x.lower().strip().split(' ')]))

In [ ]:
# save converted file
df.to_excel(r'', index=False)

In [ ]:
# update new found words to existing dictionary
new_dict = {**diss, **my_dict}

In [ ]:
# overwrite the dictionary
with open('hin_to_eng.json', "w") as outfile:
    json.dump(new_dict, outfile)